<a href="https://colab.research.google.com/github/chirayu-xx/pdf-summarizer/blob/main/pdf_summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install PyPDF2
!pip install transformers
!pip install pytesseract
!pip install PyMuPDF
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.2 MB/s eta 0:00:00


In [ ]:
 !pip install colabcode

In [5]:
!pip install fastapi==0.103.2 nest-asyncio pyngrok uvicorn
!pip install python-multipart

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 4.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 1.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.


In [6]:
!pip install sentencepiece
!ngrok authtoken 2ZLDE06RzMLQEbRfbRmZzhHTdSJ_5vnBBVzitpreaSLRbFGxD

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [7]:
from fastapi.middleware.cors import CORSMiddleware
from fastapi import FastAPI, File, UploadFile,HTTPException
from fastapi.responses import JSONResponse
from typing import List
import os
import fitz  # PyMuPDF library
from transformers import AutoTokenizer, AutoModelWithLMHead

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

# Path to store uploaded files
upload_directory = '/content/uploaded_files'
text_directory = '/content/text_files'


# Home route
@app.get("/")
def read_root():
    return {"message": "Welcome to the FastAPI server!"}


@app.post("/file")
async def upload_file(file: UploadFile = File(...)):
    # Do here your stuff with the file
    return {"filename": file.filename}

@app.post("/extract-text")
async def extract_text_from_pdf(file: UploadFile = UploadFile(...)):
    try:
        # Read PDF content
        pdf_content = await file.read()

        # Open PDF with PyMuPDF
        pdf_document = fitz.open("pdf", pdf_content)

        # Initialize an empty string to store extracted text
        text = ""

        # Iterate through pages and extract text
        for page_number in range(pdf_document.page_count):
            page = pdf_document[page_number]
            text += page.get_text()

        # Close the PDF document
        pdf_document.close()

        return JSONResponse(content={"extracted_text": text}, status_code=200)

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/summarize")
async def extract_text_from_pdf(file: UploadFile = UploadFile(...)):
    try:
        # Read PDF content
        pdf_content = await file.read()

        # Open PDF with PyMuPDF
        pdf_document = fitz.open("pdf", pdf_content)

        # Initialize an empty string to store extracted text
        text = ""

        # Iterate through pages and extract text
        for page_number in range(pdf_document.page_count):
            page = pdf_document[page_number]
            text += page.get_text()

        # Initialize the model and tokenizer
        tokenizer=AutoTokenizer.from_pretrained('T5-base')
        model=AutoModelWithLMHead.from_pretrained('T5-base', return_dict=True)

        # Encode the text
        inputs = tokenizer.encode("summarize: " + text, return_tensors="pt",truncation=True)

        # Generate the summary
        outputs = model.generate(inputs,
        max_length=1000, min_length=100)

        # Decode the summary
        summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

        return JSONResponse(content={"summary": summary}, status_code=200)

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# Example Usage:
# Send a PDF file to http://your-fastapi-host/extract-text using your frontend.


In [9]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn


# specify a port
port = 8000
ngrok_tunnel = ngrok.connect(port)

# where we can visit our fastAPI app
print('Public URL:', ngrok_tunnel.public_url)


nest_asyncio.apply()

# finally run the app
uvicorn.run(app, port=port)


Public URL: https://589d-35-237-10-17.ngrok-free.app


INFO:     Started server process [191]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [191]
